In [1]:
import math
import random
import pandas as pd
import numpy as np

from mesa import Agent
from mesa import Model
from mesa.datacollection import DataCollector
from mesa.space import Grid
from mesa.time import RandomActivation

In [326]:
def create_tx(from_address, nonce, start, end, gas_price, num_bytes):
    
    tx = {}
        
    tx['nonce'] = nonce
    tx['from_address'] = from_address # << Sensor ID
    tx['start_sync'] = start
    tx['end_sync'] = end
    tx['gas_price'] = gas_price
    # self.gas_limit = gas_limit # << unused parameter
    tx['num_bytes'] = num_bytes

    return tx

In [386]:
class Sensor(Agent):
    
    def __init__(self, unique_id, battery_life, 
                record_cost, record_freq, record_bytes, 
                compute_cost_per_byte, info_reduction,
                sign_cost,
                transmit_cost_per_byte, transmit_freq,
                gas_price, blockchain, 
                model):
        
        super().__init__(unique_id, model)
        
        if self.model.verbose:
            print('Creating Sensor agent ID', unique_id)
           
#         self.unique_id = unique_id
        self.battery_life = battery_life
        self.dead = False
        self.record_cost = record_cost
        self.record_freq = record_freq
        self.record_bytes = record_bytes
        self.compute_cost_per_byte = compute_cost_per_byte
        self.info_reduction = info_reduction
        self.sign_cost = sign_cost
        self.transmit_cost_per_byte = transmit_cost_per_byte
        self.transmit_freq = transmit_freq
        self.gas_price = gas_price
        self.blockchain = blockchain
        
        self.gwei_spent = 0
        self.data_collected = 0
        self.last_sync = 0
        self.nonce = 0
        self.db = np.array([]) # << bytes recorded per tick
        
        self.blockchain.chain.loc[0, self.unique_id] = False

    
    def record(self):

        if self.model.schedule.steps % self.record_freq == 0:
            
            self.battery_life -= self.record_cost
            self.db = np.append(self.db, self.record_bytes)
            
        else:
            self.db = np.append(self.db, 0)

    
    def compute(self, num_bytes):
        
        # Only invoked from within the transmit() method
        
        if self.info_reduction is not 1:
            self.battery_life -= self.compute_cost_per_byte * num_bytes
            return math.ceil(self.info_reduction * num_bytes)
        else:
            return num_bytes
            # with no compute cost
    
    def sign(self):
        
        # Only invoked from within the transmit() method
        self.battery_life -= self.sign_cost
    
    def next_nonce(self):
        self.nonce += 1
        return self.nonce
    
    def transmit(self):
        # Prepare data for transmission:
        
        # Calculate number of bytes to transmit (result of edge computation)
        bytes_collected = np.sum(self.db[self.last_sync : ])
        num_bytes_to_transmit = self.compute(bytes_collected)
        
        tx = create_tx(self.unique_id, self.next_nonce(), 
                       self.last_sync, self.model.schedule.steps, 
                       self.gas_price, num_bytes_to_transmit)
        
        # Prepare and sign tx
        self.sign()
        
        # Transmit, subtracting energy cost and adding gwei cost
        self.battery_life -= self.transmit_cost_per_byte * num_bytes_to_transmit
        self.blockchain.add_to_mempool(tx)
        
        self.last_sync = self.model.schedule.steps
    
    def confirm_tx(self, tx):
        
        self.gwei_spent += tx.gas_spend
        

    def step(self):
        
        if not self.dead:
            
            if self.battery_life < 0 and self.battery_life is not -999:
                if self.model.verbose:
                    print("Sensor", self.unique_id, "out of battery at tick", self.model.schedule.steps)
                self.dead = True
                pass

            self.record()    

            if self.transmit_freq >= 1:
                if self.model.schedule.steps % self.transmit_freq == 0:
                    print("TRansmitting", self.model.schedule.steps,'%', self.transmit_freq, "==", self.model.schedule.steps % self.transmit_freq)
                    self.transmit()
            elif self.transmit_freq > random.random():
                self.transmit()



In [385]:
class Blockchain(Agent):
    
    def __init__(self, unique_id, gas_price, block_gas_limit,
                gas_per_byte, gas_per_second, model):
        
        super().__init__(unique_id, model)
        
        if self.model.verbose:
            print("Blockchain created: ID", unique_id)
        
#         self.unique_id = unique_id
        self.gas_price = gas_price
        self.block_gas_limit = block_gas_limit
        self.gas_per_byte = gas_per_byte
        self.gas_per_second = gas_per_second
        self.chain = pd.DataFrame()
        
        self.tx_ct = 0
        self.mempool = pd.DataFrame(columns=["from_address", "nonce", 
                                             "start_sync", "end_sync", 
                                             "gas_price", "num_bytes", 
                                             "gas_spend", "tx_id",
                                             "mined", "block_submitted"])

    def add_to_mempool(self, tx):
        tx['gas_spend'] = tx['gas_price'] * self.gas_per_byte * tx['num_bytes']

        tx['tx_id'] = self.tx_ct
        tx['mined'] = False
        tx['block_submitted'] = self.model.schedule.steps
        row = pd.DataFrame(tx, index = [self.tx_ct])
        
        self.tx_ct += 1
        self.mempool = self.mempool.append(row, ignore_index=True)
    
    def write_data(self, num_bytes):

        gwei_spent = self.gas_per_byte * num_bytes * self.gas_price
        return gwei_spent

    def mine_block(self):
        
        print("BLOCK NUMBER:", self.model.schedule.steps)
        self.chain.loc[self.model.schedule.steps] = [False for col in self.chain.columns]

        # Sort mempool to get highest-value transactions
        mp = self.mempool[self.mempool['mined'] == False].sort_values(by=['gas_spend']).reset_index()
        
        if len(mp) > 0:
            
            mp['cum_gas'] = mp['gas_spend'].cumsum()     
            if mp['cum_gas'].max() > self.block_gas_limit:
                # If we cannot include all transactions in a block, fit as many as possible ...
                tx_mined = mp[0 : mp[mp['cum_gas'] > self.block_gas_limit].index[0]]
            else:
                tx_mined = mp[0 : ]
                
#             print('Tx to mine:', tx_mined)
            print("Mining", len(tx_mined), "out of", len(mp), "unvalidated transactions.")
            print("Gas value:", tx_mined['gas_spend'].sum())
            
            for tx in tx_mined.iterrows():
                
                print("Mining tx id:", tx[1].tx_id)
                self.model.schedule._agents[tx[1].from_address].confirm_tx(tx[1])
                self.mempool.loc[self.mempool['tx_id'] == tx[1].tx_id, "mined"] = True
                self.mempool.loc[self.mempool['tx_id'] == tx[1].tx_id, "block_mined"] = self.model.schedule.steps
                
                self.chain.loc[tx[1].start_sync : tx[1].end_sync, tx[1].from_address] = True
                
        else:
            print('Empty mempool')
            pass
        
    
    # Not used:
    def compute(self, num_seconds):
        gwei_spent = self.gas_per_second * num_seconds * self.gas_price
        return gwei_spent
    

In [387]:
class SensorBlockchainNetwork(Model):
    
    def __init__(self, num_sensors,
                verbose=True):
        
        super().__init__()
        
        self.verbose = verbose
        if self.verbose:
            print('Verbose model')
        
        self.running = True
        self.schedule = RandomActivation(self)
        self.datacollector = DataCollector(
                                model_reporters = {
                                    "active_sensors": lambda m: len(m.schedule._agents),
                                    # chain size
                                    # information availability
                                    # live sensor proportion 
                                },
                                agent_reporters = {
                                    "gwei_spent": lambda a: a.gwei_spent,
                                    "battery_life": lambda a: a.battery_life,
                                    "data_collected": lambda a: a.data_collected
                                })
        
        self.blockchain = Blockchain(self.next_id(), 10, 2000000,
                                    625, 75000000, self) 
                                    # calculated based on 
                                    # https://hackernoon.com/ether-purchase-power-df40a38c5a2f
        
        for i in range(num_sensors):
            sensor = Sensor(self.next_id(), 1000, 
                            1, 3, 32,
                            1, 1, 0.1,
                            1, 7, 20,
                            self.blockchain,
                            self)
            
            self.schedule.add(sensor)
    
    
        # Mine genesis block
        self.blockchain.chain.loc[1] = [False for col in self.blockchain.chain.columns]

        if self.verbose:
            print(num_sensors, "instantiated and added to schedule.")
    
    def step(self):
        self.schedule.step()
        if self.verbose:
            print("Mining block:", self.schedule.steps)
        if self.schedule.steps > 1:
            self.blockchain.mine_block()
        self.datacollector.collect(self)

In [401]:
model = SensorBlockchainNetwork(10, False)

In [402]:
for i in range(100):
    model.step()

TRansmitting 0 % 7 == 0
TRansmitting 0 % 7 == 0
TRansmitting 0 % 7 == 0
TRansmitting 0 % 7 == 0
TRansmitting 0 % 7 == 0
TRansmitting 0 % 7 == 0
TRansmitting 0 % 7 == 0
TRansmitting 0 % 7 == 0
TRansmitting 0 % 7 == 0
TRansmitting 0 % 7 == 0
BLOCK NUMBER: 2
Tx to mine:    index block_submitted end_sync from_address gas_price  gas_spend  mined  \
0      0               0        0            4        20   400000.0  False   
1      1               0        0            8        20   400000.0  False   
2      2               0        0            9        20   400000.0  False   
3      3               0        0           10        20   400000.0  False   
4      4               0        0            6        20   400000.0  False   

  nonce  num_bytes start_sync tx_id    cum_gas  
0     1       32.0          0     0   400000.0  
1     1       32.0          0     1   800000.0  
2     1       32.0          0     2  1200000.0  
3     1       32.0          0     3  1600000.0  
4     1       32.0

BLOCK NUMBER: 26
Tx to mine:    index  block_mined block_submitted end_sync from_address gas_price  \
0     33          NaN              21       21            2        20   

   gas_spend  mined nonce  num_bytes start_sync tx_id    cum_gas  
0  1200000.0  False     4       96.0         14    33  1200000.0  
Mining 1 out of 7 unvalidated transactions.
Gas value: 1200000.0
Mining tx id: 33
BLOCK NUMBER: 27
Tx to mine:    index  block_mined block_submitted end_sync from_address gas_price  \
0     34          NaN              21       21            4        20   

   gas_spend  mined nonce  num_bytes start_sync tx_id    cum_gas  
0  1200000.0  False     4       96.0         14    34  1200000.0  
Mining 1 out of 6 unvalidated transactions.
Gas value: 1200000.0
Mining tx id: 34
BLOCK NUMBER: 28
Tx to mine:    index  block_mined block_submitted end_sync from_address gas_price  \
0     35          NaN              21       21            9        20   

   gas_spend  mined nonce  num_bytes sta

Tx to mine:    index  block_mined block_submitted end_sync from_address gas_price  \
0     60          NaN              42       42            6        20   

   gas_spend  mined nonce  num_bytes start_sync tx_id    cum_gas  
0  1200000.0  False     7       96.0         35    60  1200000.0  
Mining 1 out of 10 unvalidated transactions.
Gas value: 1200000.0
Mining tx id: 60
BLOCK NUMBER: 49
Tx to mine:    index  block_mined block_submitted end_sync from_address gas_price  \
0     61          NaN              42       42           10        20   

   gas_spend  mined nonce  num_bytes start_sync tx_id    cum_gas  
0  1200000.0  False     7       96.0         35    61  1200000.0  
Mining 1 out of 9 unvalidated transactions.
Gas value: 1200000.0
Mining tx id: 61
TRansmitting 49 % 7 == 0
TRansmitting 49 % 7 == 0
TRansmitting 49 % 7 == 0
TRansmitting 49 % 7 == 0
TRansmitting 49 % 7 == 0
TRansmitting 49 % 7 == 0
TRansmitting 49 % 7 == 0
TRansmitting 49 % 7 == 0
TRansmitting 49 % 7 == 0
TRansmi

Tx to mine:    index  block_mined block_submitted end_sync from_address gas_price  \
0     79          NaN              49       49           11        20   

   gas_spend  mined nonce  num_bytes start_sync tx_id    cum_gas  
0  1200000.0  False     8       96.0         42    79  1200000.0  
Mining 1 out of 21 unvalidated transactions.
Gas value: 1200000.0
Mining tx id: 79
BLOCK NUMBER: 73
Tx to mine:    index  block_mined block_submitted end_sync from_address gas_price  \
0     90          NaN              63       63            8        20   

   gas_spend  mined nonce  num_bytes start_sync tx_id    cum_gas  
0  1200000.0  False    10       96.0         56    90  1200000.0  
Mining 1 out of 20 unvalidated transactions.
Gas value: 1200000.0
Mining tx id: 90
BLOCK NUMBER: 74
Tx to mine:    index  block_mined block_submitted end_sync from_address gas_price  \
0     91          NaN              63       63           10        20   

   gas_spend  mined nonce  num_bytes start_sync tx_id  

Tx to mine:    index  block_mined block_submitted end_sync from_address gas_price  \
0    122          NaN              84       84           10        20   

   gas_spend  mined nonce  num_bytes start_sync tx_id    cum_gas  
0  1200000.0  False    13       96.0         77   122  1200000.0  
Mining 1 out of 8 unvalidated transactions.
Gas value: 1200000.0
Mining tx id: 122


In [403]:
model.blockchain.mempool

,block_mined,block_submitted,end_sync,from_address,gas_price,gas_spend,mined,nonce,num_bytes,start_sync,tx_id
0,2.0,0,0,4,20,400000.0,True,1,32.0,0,0
1,2.0,0,0,8,20,400000.0,True,1,32.0,0,1
2,2.0,0,0,9,20,400000.0,True,1,32.0,0,2
3,2.0,0,0,10,20,400000.0,True,1,32.0,0,3
4,2.0,0,0,6,20,400000.0,True,1,32.0,0,4
5,3.0,0,0,7,20,400000.0,True,1,32.0,0,5
6,3.0,0,0,3,20,400000.0,True,1,32.0,0,6
7,3.0,0,0,5,20,400000.0,True,1,32.0,0,7
8,3.0,0,0,11,20,400000.0,True,1,32.0,0,8
9,3.0,0,0,2,20,400000.0,True,1,32.0,0,9


In [368]:
model.blockchain.mempool[model.blockchain.mempool['from_address'] == 2]

,block_mined,block_submitted,end_sync,from_address,gas_price,gas_spend,mined,nonce,num_bytes,start_sync,tx_id
1,2.0,0,0,2,20,400000.0,True,1,32.0,0,1
12,10.0,7,7,2,20,1200000.0,True,2,96.0,0,12


In [384]:
model.blockchain.chain

,2,3,4,5,6,7,8,9,10,11
0,True,True,True,True,True,True,True,True,True,True
1,True,True,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True,True,True
3,True,True,True,True,True,True,True,True,True,True
4,True,True,True,True,True,True,True,True,True,True
5,True,True,True,True,True,True,True,True,True,True
6,True,True,True,True,True,True,True,True,True,True
7,True,True,True,True,True,True,True,True,True,True
8,True,True,True,True,True,True,True,True,True,True
9,True,True,True,True,True,True,True,True,True,True


In [396]:
a_df = model.datacollector.get_agent_vars_dataframe()
m_df = model.datacollector.get_model_vars_dataframe()

In [398]:
m_df

,active_sensors
0,10
1,10
2,10
3,10
4,10
5,10
6,10
7,10
8,10
9,10


In [400]:
a_df.tail(100)

gwei_spent  battery_life  data_collected
Step AgentID                                          
91   2        10800000.0         -86.3               0
     3        12000000.0         -86.3               0
     4        12000000.0         -86.3               0
     5        10800000.0         -86.3               0
     6        10800000.0         -86.3               0
     7        10800000.0         -86.3               0
     8        10800000.0         -86.3               0
     9        12000000.0         -86.3               0
     10       12000000.0         -86.3               0
     11       10800000.0         -86.3               0
92   2        10800000.0         -86.3               0
     3        12000000.0         -86.3               0
     4        12000000.0         -86.3               0
     5        10800000.0         -86.3               0
     6        10800000.0         -86.3               0
     7        10800000.0         -86.3               0
     8        12000000.0         -86.3               0
     9        12000000.0         -86.3               0
     10       12000000.0         -86.3               0
     11       10800000.0         -86.3               0
93   2        10800000.0         -86.3               0
     3        12000000.0         -86.3               0
     4        12000000.0         -86.3               0
     5        10800000.0         -86.3               0
     6        12000000.0         -86.3               0
     7        10800000.0         -86.3               0
     8        12000000.0         -86.3               0
     9        12000000.0         -86.3               0
     10       12000000.0         -86.3               0
     11       10800000.0         -86.3               0
...                  ...           ...             ...
98   2        13200000.0         -86.3               0
     3        12000000.0         -86.3               0
     4        12000000.0         -86.3               0
     5        12000000.0         -86.3               0
     6        12000000.0         -86.3               0
     7        12000000.0         -86.3               0
     8        12000000.0         -86.3               0
     9        12000000.0         -86.3               0
     10       12000000.0         -86.3               0
     11       12000000.0         -86.3               0
99   2        13200000.0         -86.3               0
     3        12000000.0         -86.3               0
     4        12000000.0         -86.3               0
     5        12000000.0         -86.3               0
     6        12000000.0         -86.3               0
     7        13200000.0         -86.3               0
     8        12000000.0         -86.3               0
     9        12000000.0         -86.3               0
     10       12000000.0         -86.3               0
     11       12000000.0         -86.3               0
100  2        13200000.0         -86.3               0
     3        12000000.0         -86.3               0
     4        13200000.0         -86.3               0
     5        12000000.0         -86.3               0
     6        12000000.0         -86.3               0
     7        13200000.0         -86.3               0
     8        12000000.0         -86.3               0
     9        12000000.0         -86.3               0
     10       12000000.0         -86.3               0
     11       12000000.0         -86.3               0

[100 rows x 3 columns]

In [ ]:
# Batch Run

batch_runner = BatchRunner(
    SensorBlockchainNetwork,
    variable_params,
    fixed_params,
    iterations = 5,
    max_steps = 100,
    model_reporters = {"blockchain_size": lambda m: m.blockchain.chain_size}
)